# Exploring Pandas with some WMIC output

For this example, lets pull some current service data from our system and see how we can explore it with Pandas. Of course, for hunting or assessment, we would either have multiple instances of this output for one machine over some period of time, or we could compare the output from multiple machines.

To get the data, I just ran a `wmic SERVICE LIST BRIEF` as Admin on my personal computer. Below is the output.

In [1]:
filename = "C:\services.txt"

with open(filename) as file:
    for line in file:
        print(line.strip())        

ÿþE x i t C o d e     N a m e                                                                                                         P r o c e s s I d     S t a r t M o d e     S t a t e         S t a t u s     
 
 1 0 7 7             A J R o u t e r                                                                                                 0                     M a n u a l           S t o p p e d     O K             
 
 1 0 7 7             A L G                                                                                                           0                     M a n u a l           S t o p p e d     O K             
 
 1 0 7 7             A p p I D S v c                                                                                                 0                     M a n u a l           S t o p p e d     O K             
 
 0                   A p p i n f o                                                                                                   1 0 3 6 4 

Of course, you could use a remote wmic command or PowerShell to get a similar table from a remote machine or, for the purpose of this exploration, the local machine (if Windows).

In [65]:
import os

services = os.popen("wmic SERVICE LIST BRIEF").read().split('\n\n')
services

['ExitCode  Name                                                    ProcessId  StartMode  State    Status   ',
 '1077      AJRouter                                                0          Manual     Stopped  OK       ',
 '1077      ALG                                                     0          Manual     Stopped  OK       ',
 '1077      AppIDSvc                                                0          Manual     Stopped  OK       ',
 '0         Appinfo                                                 10364      Manual     Running  OK       ',
 '1077      AppReadiness                                            0          Manual     Stopped  OK       ',
 '0         AppXSvc                                                 13228      Manual     Running  OK       ',
 '1077      aspnet_state                                            0          Manual     Stopped  OK       ',
 '0         AudioEndpointBuilder                                    3292       Auto       Running  OK       ',
 

Next lets import pandas and get our dataframe. This first example is using the saved wmic output text file.

In [13]:
import pandas as pd

# Defining the separator as multiple spaces fixes the issues with the text 
# based table.
dta = pd.read_csv(filename, sep='\s{2,}', encoding="utf-16", engine = 'python')

A quick check of the first few lines to see what we are working with:

In [14]:
dta.head()

,ExitCode,Name,ProcessId,StartMode,State,Status
0,1077,AJRouter,0,Manual,Stopped,OK
1,1077,ALG,0,Manual,Stopped,OK
2,1077,AppIDSvc,0,Manual,Stopped,OK
3,0,Appinfo,10364,Manual,Running,OK
4,1077,AppReadiness,0,Manual,Stopped,OK


Let's see if we can do something similar with the list generated from the `popen` method above. We can see that each list entry corresponds to a row in the final dataframe we want.

In [66]:
# lets split each list entry in services
for i in range(len(services)):
    services[i] = services[i].split()

services


[['ExitCode', 'Name', 'ProcessId', 'StartMode', 'State', 'Status'],
 ['1077', 'AJRouter', '0', 'Manual', 'Stopped', 'OK'],
 ['1077', 'ALG', '0', 'Manual', 'Stopped', 'OK'],
 ['1077', 'AppIDSvc', '0', 'Manual', 'Stopped', 'OK'],
 ['0', 'Appinfo', '10364', 'Manual', 'Running', 'OK'],
 ['1077', 'AppReadiness', '0', 'Manual', 'Stopped', 'OK'],
 ['0', 'AppXSvc', '13228', 'Manual', 'Running', 'OK'],
 ['1077', 'aspnet_state', '0', 'Manual', 'Stopped', 'OK'],
 ['0', 'AudioEndpointBuilder', '3292', 'Auto', 'Running', 'OK'],
 ['0', 'Audiosrv', '3480', 'Auto', 'Running', 'OK'],
 ['1077', 'autotimesvc', '0', 'Manual', 'Stopped', 'OK'],
 ['1077', 'AxInstSV', '0', 'Manual', 'Stopped', 'OK'],
 ['0', 'BDESVC', '0', 'Manual', 'Stopped', 'OK'],
 ['0', 'BFE', '3076', 'Auto', 'Running', 'OK'],
 ['0', 'BITS', '812', 'Auto', 'Running', 'OK'],
 ['0', 'BrokerInfrastructure', '880', 'Auto', 'Running', 'OK'],
 ['1077', 'Browser', '0', 'Manual', 'Stopped', 'OK'],
 ['0', 'BTAGService', '1436', 'Manual', 'Running'

In [69]:
data = pd.DataFrame(services)

data.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,ExitCode,Name,ProcessId,StartMode,State,Status,None,None,None,None,None,None
1,1077,AJRouter,0,Manual,Stopped,OK,None,None,None,None,None,None
2,1077,ALG,0,Manual,Stopped,OK,None,None,None,None,None,None
3,1077,AppIDSvc,0,Manual,Stopped,OK,None,None,None,None,None,None
4,0,Appinfo,10364,Manual,Running,OK,None,None,None,None,None,None


In [70]:
# first we'll drop the extra columns and any rows with no data

data = data.drop(axis=1, columns = range(6,12)).dropna()

data

,0,1,2,3,4,5
0,ExitCode,Name,ProcessId,StartMode,State,Status
1,1077,AJRouter,0,Manual,Stopped,OK
2,1077,ALG,0,Manual,Stopped,OK
3,1077,AppIDSvc,0,Manual,Stopped,OK
4,0,Appinfo,10364,Manual,Running,OK
...,...,...,...,...,...,...
272,1077,PrintWorkflowUserSvc_953d5,0,Manual,Stopped,OK
273,1077,UdkUserSvc_953d5,0,Manual,Stopped,OK
274,1077,UnistoreSvc_953d5,0,Manual,Stopped,OK
275,1077,UserDataSvc_953d5,0,Manual,Stopped,OK


In [71]:
columns = data.iloc[0]
data = data[1:]
data.columns = columns

data

0     ExitCode
1         Name
2    ProcessId
3    StartMode
4        State
5       Status
Name: 0, dtype: object


,ExitCode,Name,ProcessId,StartMode,State,Status
1,1077,AJRouter,0,Manual,Stopped,OK
2,1077,ALG,0,Manual,Stopped,OK
3,1077,AppIDSvc,0,Manual,Stopped,OK
4,0,Appinfo,10364,Manual,Running,OK
5,1077,AppReadiness,0,Manual,Stopped,OK
...,...,...,...,...,...,...
272,1077,PrintWorkflowUserSvc_953d5,0,Manual,Stopped,OK
273,1077,UdkUserSvc_953d5,0,Manual,Stopped,OK
274,1077,UnistoreSvc_953d5,0,Manual,Stopped,OK
275,1077,UserDataSvc_953d5,0,Manual,Stopped,OK


Now lets take a look at some basic stats on the dataframe. Remember that by default, the `describe()` method will only show stats for numerical fields. To show stats for categorical data, we can use this little loop:

In [4]:
for col in dta.columns:
    dta[col].describe()

**Note:** If that didn't work, run the following cell and then re-run the last one. Notebooks do not display multiple outputs by default.

In [5]:
from IPython.core.interactiveshell import InteractiveShell

# pretty print all cell's output and not just the last one
InteractiveShell.ast_node_interactivity = "all"

If we are potentially going to compare this output over time or against other machines or a baseline, lets add the hostname and a time stamp to our records. To have a clean and usable dataframe, it is important that each record contains all of the pertinent information. If we want to compare the time of day the a process is running against which hosts are running the process, then each individual process record must contain all of that information.

The next line uses the native Windows commandline to get the hostname. For most applications, this will not be viable, since the data should come from mission-related systems and not from the host running the notebook server. You could potentially get the hostname from as asset inventory, file naming convention, etc.

In [6]:
import os

hostname = os.popen('hostname').read().strip()
hostname

'LAPTOP-06RQRGUU'

Now we can use the insert method to add our hostname column. The first argument is the index for the new column. Next we have the column heading. Finally, we add the hostname string we grabbed in the last cell.


In [7]:
dta.insert(0, 'Hostname', hostname)
dta.head()

,Hostname,ExitCode,Name,ProcessId,StartMode,State,Status
0,LAPTOP-06RQRGUU,1077,AJRouter,0,Manual,Stopped,OK
1,LAPTOP-06RQRGUU,1077,ALG,0,Manual,Stopped,OK
2,LAPTOP-06RQRGUU,1077,AppIDSvc,0,Manual,Stopped,OK
3,LAPTOP-06RQRGUU,0,Appinfo,10364,Manual,Running,OK
4,LAPTOP-06RQRGUU,1077,AppReadiness,0,Manual,Stopped,OK


One last useful bit of information is a time stamp. Pandas has a native datatype of Timestamp we can use, along with the system time for this example. 

First let's take a look at what datetime can provide.

In [73]:
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%m/%d/%Y %H:%M:%S")
dt_string

'11/21/2020 13:14:00'

Now lets insert that datetime into our DataFrame:

In [74]:
time = pd.Timestamp(now)

dta.insert(0, "Time", time)
dta.head()

,Time,ExitCode,Name,ProcessId,StartMode,State,Status
0,2020-11-21 13:14:00.167826,1077,AJRouter,0,Manual,Stopped,OK
1,2020-11-21 13:14:00.167826,1077,ALG,0,Manual,Stopped,OK
2,2020-11-21 13:14:00.167826,1077,AppIDSvc,0,Manual,Stopped,OK
3,2020-11-21 13:14:00.167826,0,Appinfo,10364,Manual,Running,OK
4,2020-11-21 13:14:00.167826,1077,AppReadiness,0,Manual,Stopped,OK


With data like this, we can now combine this dataframe with other ones collected independently and look for insights across them all.

In [75]:
dta.dtypes

Time         datetime64[ns]
ExitCode              int64
Name                 object
ProcessId             int64
StartMode            object
State                object
Status               object
dtype: object